# CONSTRUCCIÓN DE LOS "FOOTPRINTS"

Construimos los footprints temporales de cada cliente a partir de sus transacciones, de tal manera que agrupamos en un intervalo de d días.


Dado:

    Sesion de TX
        s = {cliente, timestamp, Monto}
    
    Cada cliente tiene una secuencia de sesiones de TX (S)
        S = { s1, s2 , s3, ..., sn}                n: # de sesiones de cada cliente
        
        

In [1]:
# LIBRERIAS
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys

## Preparacion de datos

### Cargando datos

In [2]:
data_original = pd.read_csv("./data/muestra.csv", encoding = "iso-8859-1")
data_original.head()

,CO_ID,BILLCYCLE,NUMCODPLANTARIFARIO,F_ACT,F_TRAFICO,HORA,GB_TOTAL,NO_FREE,FREE,BANCOS,...,SPOTIFY_PI,APPLEM_PI,SOUNDC_PI,FACE_PI,MAIL_PI,WHATS_RO_I,WHATS_RO_IF,WAZE_RO,EMAIL_RO,WHARS_RO2
0,27782393,3.0,1981,11/11/2016,02/08/2018,Noche,0.0000,0.0000,0.0000,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0,0.0
1,41901365,5.0,1982,20/01/2018,02/08/2018,Mañana,0.0411,0.0301,0.0007,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0,0.0
2,31801426,5.0,1981,17/04/2017,02/08/2018,Madrugada,0.0138,0.0078,0.0014,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0,0.0
3,41410806,4.0,1982,05/01/2018,02/08/2018,Madrugada,0.0799,0.0603,0.0078,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0,0.0
4,42886287,5.0,1981,20/02/2018,02/08/2018,Mañana,0.2182,0.0096,0.0000,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0,0.0


### Preparacion de datos

In [3]:
data = data_original[['CO_ID','BILLCYCLE','F_TRAFICO','HORA','GB_TOTAL']]
data['año'] = data['F_TRAFICO'].apply(lambda fecha: int(fecha[6:]))
data['mes'] = data['F_TRAFICO'].apply(lambda fecha: int(fecha[4:5]))
data['dia'] = data['F_TRAFICO'].apply(lambda fecha: int(fecha[1:2]))

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,CO_ID,BILLCYCLE,F_TRAFICO,HORA,GB_TOTAL,año,mes,dia
0,27782393,3.0,02/08/2018,Noche,0.0000,2018,8,2
1,41901365,5.0,02/08/2018,Mañana,0.0411,2018,8,2
2,31801426,5.0,02/08/2018,Madrugada,0.0138,2018,8,2
3,41410806,4.0,02/08/2018,Madrugada,0.0799,2018,8,2
4,42886287,5.0,02/08/2018,Mañana,0.2182,2018,8,2


### Limpiza de datos]

In [4]:
data[data.isnull().any(axis=1)].head(5)

,CO_ID,BILLCYCLE,F_TRAFICO,HORA,GB_TOTAL,año,mes,dia
62,45866282,NaN,02/08/2018,Madrugada,0.3994,2018,8,2
120,45119243,NaN,04/08/2018,Noche,0.4427,2018,8,4
213,44466028,NaN,01/08/2018,Tarde,0.0118,2018,8,1
324,44466625,NaN,02/08/2018,Noche,0.0785,2018,8,2
471,45866282,NaN,02/08/2018,Noche,0.1304,2018,8,2


In [5]:
data = data.fillna(9)
data.head(5)

## Definicion de variables

### Billcycle

In [6]:
# DEFINIMOS Billcycles: PARA CREAR UN FOOTPRINT DINAMICO
billcycles = ((data.groupby(['BILLCYCLE'], as_index=False, sort=True)).count())['BILLCYCLE']
billcycles = list(map(int, billcycles))
billcycles

[1, 3, 4, 5, 9]

### Clientes

In [7]:
# DEFINIMOS LA LISTA DE CLIENTES
clientes =  data.groupby('CO_ID').CO_ID.count().index
clientes

Int64Index([ 2581717,  3132200,  3273844,  3478078,  3768670,  3877738,
             3903875,  3952329,  3954876,  3956151,
            ...
            46812929, 46813575, 46817874, 46822457, 46832405, 46842470,
            46845419, 46845790, 46853110, 46856084],
           dtype='int64', name='CO_ID', length=40059)

# FOOTPRINT PARA CADA MCCG

## Unidad de TXs temporales (U)

Resume un conjunto de TXs en un periodo de tiempo **“tau”**.
<div>Los footprints que creamos será la agrupación de todas las transacciones en "tau" de cada cliente, y representará una unidad de comportamiento con 3 dimensiones.</div> 


Donde:
1.  **tau** = 1 semana
2.  dimensión 1:    **d**   = 7 días
3.  dimensión 2:	**t**   = 4 turnos	 
4.  dimensión 3:    **m**   = # de mccgs en nuestra data.

    
Para cada cliente:
    
    Entrada:   S = {s1, ..., sn}
    Salida:    SS = {U(1), U(2), U(3), ...,U(m)}     donde m <=n

### Funciones de apoyo

Definimos funcion para generar los footprint (U) de un usuario, con los parametros:

1. <div> **user**:  Dataset original filtrado para un cliente determinado</div> 
2. <div> **monto**: Indica si los footprint seran generados con la suma de los el numero de TXs (False) o la suma de los montos por cada TX (True)</div> 

In [8]:
def procesar_u(user, tipo_eth = False):    
    uid=list(user['CO_ID'])[0]              # Cliente_id
    years = set(list(user['año']))              # Lista los años en que se tiene TXs registradas
    anni = {year:{} for year in list(years)}    # definimos anni como una lista 
    
    # para cada fila, es decir, cada TXs del cliente)
    for dat in  range(0,len(user)):
        año = user.iloc[dat]['año']
        mes = user.iloc[dat]['mes']
        dia = user.iloc[dat]['dia']
        week=str(datetime.datetime(año,mes,dia).isocalendar()[1])
        if len(week)==1:
            week = '0' + week
        weekday=datetime.datetime(año,mes,dia).weekday()
        turn = user.iloc[dat]['HORA']
        billcycle = user.iloc[dat]['BILLCYCLE']
        
        # Si la semana no existe en el año
        if not(week in anni[año]):
            anni[año][week] = {}
        # Si el billcycle no existe en la semana y año
        if not (billcycle in anni[año][week]):
            anni[año][week][billcycle]={}  #NUMERO DE MCCGs VARIABLES
        # Si el turno no existe en el mccg,semana y año
        if not (turn in anni[año][week][billcycle]):
            anni[año][week][billcycle][turn]=np.array([0]*7)  #CUATRO TURNOS
            
        if tipo_eth:
            anni[año][week][billcycle][turn][weekday]+=user.iloc[dat]['GB_TOTAL'].astype(float)
        else:
            anni[año][week][billcycle][turn][weekday]+=1 
            
    return uid,anni

### Procesando U

Generamos en formato json el footprint de cada cliente
donde el los índices son: **cliente_id**, **año**, **semana**, **mccg**, **turno** conteniendo un **array[0,...,6]** con los dias.



In [9]:
##################################################
#        Procesando U de cada CLIENTE
##################################################
    
profiles={}           # Creamos lista de prefiles
contador=0 
print("Number of rows "+str(len(data))) 

# Para cada cliente
for cliente in clientes:
    cliente_i= data[data['CO_ID'] == cliente]       # filtramos dataset solo para el cliente i
    results=procesar_u(cliente_i, tipo_eth=False)          # procesamos u del usuario i
    profiles[results[0]]=results[1]                     # cargamos lista de indice "uid" con la data del cliente(json)
    contador += 1
    if contador % 100000 == 1:
        print("vamos en el ",contador)

Number of rows 108587
vamos en el  1


In [10]:
# profiles

Creamos la cabecera dinámica donde se guardaran todos los footprints generados

In [11]:
cabecera = 'CO_ID,YEAR,WEEK,PROFILE_ID,BILLCYCLE,TURN,SIZE'
for i in range(len(billcycles)):
    for j in range(4):                # numero de turnos
        for k in range(7):            # numero de dias
            cabecera = cabecera+','+'b'+str(billcycles[i])+'t'+str(j)+'d'+str(k)
cabecera = cabecera+'\n'
cabecera

'CO_ID,YEAR,WEEK,PROFILE_ID,BILLCYCLE,TURN,SIZE,b1t0d0,b1t0d1,b1t0d2,b1t0d3,b1t0d4,b1t0d5,b1t0d6,b1t1d0,b1t1d1,b1t1d2,b1t1d3,b1t1d4,b1t1d5,b1t1d6,b1t2d0,b1t2d1,b1t2d2,b1t2d3,b1t2d4,b1t2d5,b1t2d6,b1t3d0,b1t3d1,b1t3d2,b1t3d3,b1t3d4,b1t3d5,b1t3d6,b3t0d0,b3t0d1,b3t0d2,b3t0d3,b3t0d4,b3t0d5,b3t0d6,b3t1d0,b3t1d1,b3t1d2,b3t1d3,b3t1d4,b3t1d5,b3t1d6,b3t2d0,b3t2d1,b3t2d2,b3t2d3,b3t2d4,b3t2d5,b3t2d6,b3t3d0,b3t3d1,b3t3d2,b3t3d3,b3t3d4,b3t3d5,b3t3d6,b4t0d0,b4t0d1,b4t0d2,b4t0d3,b4t0d4,b4t0d5,b4t0d6,b4t1d0,b4t1d1,b4t1d2,b4t1d3,b4t1d4,b4t1d5,b4t1d6,b4t2d0,b4t2d1,b4t2d2,b4t2d3,b4t2d4,b4t2d5,b4t2d6,b4t3d0,b4t3d1,b4t3d2,b4t3d3,b4t3d4,b4t3d5,b4t3d6,b5t0d0,b5t0d1,b5t0d2,b5t0d3,b5t0d4,b5t0d5,b5t0d6,b5t1d0,b5t1d1,b5t1d2,b5t1d3,b5t1d4,b5t1d5,b5t1d6,b5t2d0,b5t2d1,b5t2d2,b5t2d3,b5t2d4,b5t2d5,b5t2d6,b5t3d0,b5t3d1,b5t3d2,b5t3d3,b5t3d4,b5t3d5,b5t3d6,b9t0d0,b9t0d1,b9t0d2,b9t0d3,b9t0d4,b9t0d5,b9t0d6,b9t1d0,b9t1d1,b9t1d2,b9t1d3,b9t1d4,b9t1d5,b9t1d6,b9t2d0,b9t2d1,b9t2d2,b9t2d3,b9t2d4,b9t2d5,b9t2d6,b9t3d0,b9t3d1,b9t3d2,

In [12]:
def turnos_def(x):
    res = -1
    if x == "Madrugada":
        res = 0
    if x == "Mañana":
        res = 1
    if x == "Tarde":
        res = 2
    if x == "Noche":
        res = 3
    return res

In [13]:
outfile='./resultados/U'           # Indicamos archivo de salida 
individual_footprint="%s.footprint" %(outfile)
fw=open(individual_footprint,'w')  

fw.write(cabecera)                    # Escribimos la cabecera
footprints=0 

# Guardamos el Json como un dataframe
# Para cada uid (cliente)
for uid in profiles:   
    profile_id=0
    # En cada año
    for year in profiles[uid]:       
        # Por cada semana
        for week in profiles[uid][year]:                        
            temp=np.zeros(4*7*len(billcycles))                      # creamos tensor con 0s
            
            for billcycle in profiles[uid][year][week]:             # cargamos los mccgs
                pos_billcycle = list(billcycles).index(billcycle)             # posicion del mccg
                temp2 = np.zeros(4*7)                          # creamos vector temporal turnos por dias con 0s
                for turn in profiles[uid][year][week][billcycle]:   # cargamos turnos
                    pos_turn = turnos_def(turn)                            # posicion del turno
                    
                    d=profiles[uid][year][week][billcycle][turn]    # Dias como array        
                    
                    for k in range(pos_turn*7,(pos_turn+1)*7): # cargamos dias
                        temp2[k] += d[k-(pos_turn*7)]          # posicion de cada dia en el vector temporal
                
                for j in range(pos_billcycle*28,(pos_billcycle+1)*28):
                    temp[j] = temp2[j-(pos_billcycle*28)]           # vector temporal es cargado en el tensor
                
            # Escribimos los datos del primer comportamiento (Tensor)    
            txt = ''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','+str(billcycle)+','+str(turn)+','+str(sum(temp))
            for i in range(len(temp)):
                txt = txt +','+str(temp[i])
            fw.write(txt +'\n')

            profile_id += 1    # perfil cambia cada unidad de fecha diferente (eg. cada semana != es un perfil) de cada cliente
            footprints += 1
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))

number of footprint: 40059
